In [2]:
class Node:
    def __init__(self,position,father):
        self.position = position 
        self.father = father 
        
    def __repr__(self):
        return str(self.position[0])+" "+str(self.position[1])

#Function to create a list of nodes
def Vertices(L): 
    Vertices = []
    start = []
    goal = []
    for i in range(len(L)):
        for j in range(len(L[i])):
            Ver = []
            Ver.append(i)
            Ver.append(j)
            if L[i][j] != '%': #Wall
                Vertices.append(Ver)
                if L[i][j] == 'P': #Pacman Position
                    start = Ver[:]
                elif L[i][j] == '.': #Fruit
                    goal = Ver[:]
    return Vertices,start,goal

#Search a node in the list of vertices by coordinate (x,y)
def Search(x,y,Vert): 
    for i in Vert:
        if x == i[0] and y == i[1]:
            return i
    return False

#Generator of Graph
def Graph(Vert,L):
    G = []                       #If we search in the maze looking in the right node
    A = []                       #and the down node we can find all the possibles neighbors in the graph
    N2 = []                       
    for i in Vert:
        x,y = i[0],i[1]
        N2 = Search(x,y+1,Vert)     #Search node in the down from the current position
        if N2 != False:
            A.append(Node(i,None))
            A.append(Node(N2,None))
            #A.append(1)
            G.append(A)
        A = []
        N2 = Search(x+1,y,Vert)     #Search a node in the right of the current position
        if N2 != False:
            A.append(Node(i,None))
            A.append(Node(N2,None))
            #A.append(1)
            G.append(A)
        A = []
    return G

#Function to read input data from Hacker Rank
def inputData():
    start = input()
    goal = input()
    mazeMap = input()
    mazeMap = mazeMap.split()
    for i in range(int(mazeMap[0])):
        L.append(input())
    return start, goal, L
        
#Verify if pacman already pass on this node searching in the list of nodes
def Seen(Node,Seen):
    if len(Seen) != 0:
        for i in Seen:
            if Node.position == i.position:
                return False
    return True

#Search the neighbors of a node in the graph
def findNeighbors(actualNode,G,border):
    neighbors = []
    for i in G:
        if i[0].position == actualNode.position:
            neighbors.append(Node(i[1].position,actualNode))
        elif i[1].position == actualNode.position:
            neighbors.append(Node(i[0].position,actualNode))
    #Verificar si algun vecino ya esta en la frontera para no meterlo
    for i in neighbors:
        if Seen(i,border):
            border.append(i)
#Generate a list with the path from the start to the goal in the maze
def Path(seen,path):
    n = seen[:]
    e = n[len(n)-1]
    while e.father != None:
        e = e.father
        path.append(e)
    path.reverse()
    return path

In [5]:
def DFS(G,start,goal):
    border = []
    seen = []
    path = []
    border.append(start)
    while border:
        actualNode = border.pop(len(border)-1) #Extrack nodo from stack
        if actualNode.position == goal.position:
            seen.append(actualNode)
            path.append(actualNode)
            break
        else:
            if Seen(actualNode,seen):
                seen.append(actualNode)
                findNeighbors(actualNode,G,border)
    return seen,Path(seen,path)

def BFS(G,start,goal):
    border = []
    seen = []
    path = []
    border.append(start)
    while border:
        actualNode = border.pop(0) #Extraer nodo de la cola
        if actualNode.position == goal.position:
            seen.append(actualNode)
            path.append(actualNode)
            break
        else:
            #Checar si el nodo no se ha visitado ya para buscar sus vecinos
            if Seen(actualNode,seen):
                seen.append(actualNode)
                findNeighbors(actualNode,G,border)
    return seen,Path(seen,path)

In [6]:
if __name__ == '__main__':
    V = []
    G = []
    start = []
    goal = []
    #L = []
    input_file = 'mazeH2.txt'
    L = open(input_file).readlines()
    #start, goal, L = inputData()
    V,start,goal = Vertices(L)
    G = Graph(V,L)
    #seen,path = DFS(G,Node(start,None),Node(goal,None))
    seen,path = BFS(G,Node(start,None),Node(goal,None))
    print(len(seen))
    for i in seen:
        print(i)
    print(len(path)-1)
    for j in path:
        print(j)

90
11 9
11 8
11 10
11 7
11 11
11 6
11 12
10 6
11 13
9 6
10 13
8 6
9 5
9 7
9 13
7 6
8 5
9 4
9 8
8 13
9 12
6 6
7 5
8 4
9 3
10 4
8 8
9 9
7 13
9 11
5 6
6 5
7 4
8 3
11 4
7 8
9 10
6 13
4 6
5 5
5 7
6 4
7 3
11 3
7 9
5 13
3 6
5 4
5 8
6 3
11 2
7 10
4 13
2 6
3 5
3 7
5 3
5 9
11 1
7 11
3 13
1 6
3 4
4 3
5 2
4 9
5 10
10 1
2 13
3 12
3 14
1 7
2 4
3 3
5 1
3 9
4 10
5 11
9 1
1 13
3 15
1 8
1 4
4 1
6 1
2 9
3 10
8 1
1 12
2 15
15
11 9
11 10
11 11
11 12
11 13
10 13
9 13
8 13
7 13
6 13
5 13
4 13
3 13
3 14
3 15
2 15
